In [1]:
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
import _pickle as pkl
from numpy import *
from joblib import Parallel, delayed


In [2]:
def aa_knn(trainFeature, trainLabel, testFeature, testLabel, k):
    rows = testFeature.shape[0]
    cols = testFeature.shape[1]

    train_num = trainFeature.shape[0]
    dist = np.zeros(shape=(train_num))
    prediction = np.zeros(shape=(rows,trainLabel.shape[1]))

    for i in range(rows):
        test = testFeature[i,:]
        for j in range(train_num):
            train = trainFeature[j,:]
            V = test - train
            sqV = V**2
            sqDis = sqV.sum(axis=0)
            dist[j] = math.sqrt(sqDis)

        indice = dist.argsort()
        sort_dis = sorted(dist)
        sum_label = np.zeros(shape=trainLabel.shape[1])
        for l in range(k):
            sum_label = sum_label + trainLabel[l,:]

        prediction[i,:] = sum_label/k
    
    idx = prediction.argmax(axis=1)
    prediction = (idx[:,None] == np.arange(prediction.shape[1])).astype(int)

    score = metrics.accuracy_score(prediction, testLabel)

    return prediction, score 


In [3]:
def one_iteration(tolerance, trainFeature, trainLabel, testFeature, testLabel, k, sources=None):
    if sources is None:
        sources = {i: np.array([i]) for i in range(len(trainFeature))}
    elif not isinstance(sources, dict):
        sources = {i: np.where(sources == i)[0] for i in set(sources)}

    idxs = np.random.permutation(len(sources))
    marginal_contribs = np.zeros(len(trainFeature))

    #x_batch = np.zeros((0,trainFeature.shape[1]))
    #y_batch = np.zeros(0,float)

    x_batch = trainFeature[idxs[0]]
    y_batch = trainLabel[idxs[0]]

    truncation_counter = 0

    #源代码里面是随机抽取测试数据，进行100次实验获得到的平均socre，这里直接设置为零
    new_score = 0

    for sh, idx in enumerate(idxs):
        old_score = new_score

        #x_batch = np.concatenate(x_batch, trainLabel[sources[idx]])
        #y_batch = np.concatenate(y_batch, trainFeature[sources[idx]])
        if sh == 0 :
                a=0
        else:
            x_batch = np.vstack((x_batch, trainFeature[idx]))
            y_batch = np.vstack((y_batch, trainLabel[idx]))
        if x_batch.shape[0] >= 2:
            _,new_score = aa_knn(trainFeature, trainLabel, testFeature, testLabel, k)

        marginal_contribs[sources[idx]] = (new_score - old_score)/len(sources[idx])
        mean_score = 0.18

        distance_to_full_score = np.abs(new_score - mean_score)
        if distance_to_full_score <= tolerance * mean_score:
            truncation_counter += 1
            if truncation_counter > 5:
                break
        else:
            truncation_counter = 0
    return marginal_contribs
        

In [ ]:
def _tmc_error(mem):
    if len(mem) < 100:
        return 1.0
    all_vals = (np.cumsum(mem, 0)/np.reshape(np.arange(1, len(mem)+1), (-1,1)))[-100:]
    errors = np.mean(np.abs(all_vals[-100:] - all_vals[-1:])/(np.abs(all_vals[-1:]) + 1e-12), -1)
    return np.max(errors)

In [20]:
def run_tmc(n_job,iterations,error,tolerance, trainFeature, trainLabel, testFeature, testLabel, k, sources=None, verbose=1):
    def _parallel_unit(tolerance, trainFeature, trainLabel, testFeature, testLabel, k, sources=None):
        return one_iteration(tolerance, trainFeature, trainLabel, testFeature, testLabel, k, sources=None)
    i=1
    
    while True:
        mem_tmc = np.zeros((0, len(trainFeature)))
        mems = Parallel(n_jobs=n_job,verbose=verbose)(delayed(_parallel_unit)(tolerance, trainFeature, trainLabel, testFeature, testLabel, k, sources=None) for i in range(iterations))
        mem_tmc = np.concatenate([mem_tmc,np.array(mems)])
        err = _tmc_error(mem_tmc)
        if err < error:
            print(' cur error: {:5f} < {}.'.format(err,error))
                break
        print(' cur error: {:5f} > {}.'.format(err,error))

    return np.mean(mem_tmc,0)

In [6]:
file_feature = open("ldlFeature.txt",'r')
feature = []

for line in file_feature:
    one_fea = list(map(lambda x : float(x), line.split()))
    feature.append(one_fea)


file_label = open("binaryLabel.txt",'r')
label = []

for line in file_label:
    one_label = list(map(lambda x : float(x), line.split()))
    label.append(one_label)


trainFeature = array(feature[:100])
trainLabel = array(label[:100])

testFeature = array(feature[170:])
testLabel = array(label[170:])

In [ ]:
for i in range(1):
    new_value = run_tmc(5, 100, 0.1, trainFeature, trainLabel, testFeature, testLabel, 5, sources=None)
    
    mod = True

    trainLabel_tolist = trainLabel.tolist()
    idx_high = int(np.where(trainLabel[i]==np.max(trainLabel[i]))[0])
    idx_low = 1 - idx_high

    mod_count = 0

    while mod:
        #a = np.abs(trainLabel[i,0] - trainLabel[i,1])
        old_value = new_value

        #new_value,_ = tmc_shap(0.3, trainFeature, trainLabel, testFeature, testLabel, 5, 10, sources=None)

        trainLabel_tolist[i][idx_high] -= 0.01
        trainLabel_tolist[i][idx_low] = 1 - trainLabel_tolist[i][idx_high]

        if trainLabel_tolist[i][idx_high] <= 0:
            mod = False

        trainLabel = array(trainLabel_tolist)
        
        new_value = run_tmc(5, 100, 0.1, trainFeature, trainLabel, testFeature, testLabel, 5, sources=None)
        
        old_indxs = np.argsort(old_value)
        new_indxs = np.argsort(new_value)
        if old_indxs[i] > new_indxs[i]:
            mod = False
            print(trainLabel[i])
         


In [21]:
a = run_tmc(2,2,0.1, trainFeature, trainLabel, testFeature, testLabel, 5, sources=None)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    2.1s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    2.1s finished


In [26]:
import time

start = time.time()

Parallel(n_jobs=2)(delayed(one_iteration)(0.1, trainFeature, trainLabel, testFeature, testLabel, 5, sources=None) )

end = time.time()

TypeError: cannot unpack non-iterable function object

In [22]:
a = array([1,4,2,6])
b = np.argsort(a)
print(b)

[0 2 1 3]
